In [77]:
import os                             #### FOR MULTIPLE FEATURES X, SINGLE Y ####
import tensorflow as tf

W = tf.Variable(tf.zeros([5,1], name="weights"))
b = tf.Variable(0., name="bias")

def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])
    #filename_queue=tf.train.string_input_producer(['C:\\Users\\Thuy Nguyen\\Desktop\\Tensorflow learning\\train.csv'])
    
    reader = tf.TextLineReader(skip_header_lines=1) # define a (Excel) file reader that skips the first row (header)
    key, value = reader.read(filename_queue) # load the rest of the file to (key, value)
    
    # convert a Tensor from type string (the text line) in a tuple of tensor columns with the specified defaults,
    # which also sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults) 
    
    # load 'batch_size' rows in a single tensor
    return tf.train.shuffle_batch(decoded, batch_size=batch_size, capacity=batch_size*50, min_after_dequeue=batch_size)

In [78]:
# ------- define the training loop operations -------

def combine_inputs(X):
    #print(X) # Tensor("transpose_5:0", shape=(100, 5), dtype=float32)
    return tf.matmul(X, W) + b
    
def inference(X): # used in the evaluate() only
    return tf.sigmoid(combine_inputs(X)) # y = 1 / (1 + exp(-x)): is just to calculate a probability

def loss(X, Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(combine_inputs(X), Y))
        # With cross entropy, as the predicted probability comes closer to 0 for the “yes” example, the penalty 
        # increases closer to infinity. This makes it impossible for the model to make that misprediction after 
        # training --> cross entropy better suits as a loss function.
        # Logits means the func. operates on the unscaled output of earlier layers. Sum of the inputs may not 
        # equal 1, that the values are not probabilities (might have an input of 5).
    
    # TESTING (not working)
    #Y_pred = combine_inputs(X)
    #print(Y)              # Tensor("Reshape_4:0", shape=(100, 1), dtype=float32)
    #print(Y_pred)         # Tensor("add_6:0",     shape=(100, 1), dtype=float32) 
    #print(tf.log(Y_pred)) # Tensor("Log_3:0",     shape=(100, 1), dtype=float32)
    #Y_mul_logYpred = tf.multiply(Y, tf.log(Y_pred))
    #print(Y_mul_logYpred) # Tensor("Mul_2:0",     shape=(100, 1), dtype=float32)
    #Y_mul_logYpred_1minus = tf.multiply(1-Y, 1-tf.log(Y_pred))
    #print(Y_mul_logYpred_1minus) # Tensor("Mul_5:0", shape=(100, 1), dtype=float32)
    #result = -tf.reduce_mean(Y_mul_logYpred + Y_mul_logYpred_1minus)

def inputs():
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        read_csv(100, 'train.csv', [[0.0], [0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]])
    
    # convert categorical data
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))
    gender = tf.to_float(tf.equal(sex, ["female"]))
    
    # pack all features in a single matrix, then transpose to have a matrix of 1 sample per row & 1 feature per column
    features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100, 1])
    
    return features, survived

def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

def evaluate(sess, X, Y):
    Y_predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(Y_predicted, Y), tf.float32))))

# ---------------------------------------------------

In [79]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    # X is features of [is_first_class, is_second_class, is_third_class, gender, age]
    # Y is training_output of [survived], it has 0 and 1 values only
    X, Y = inputs()
    #print(sess.run(X)) # USE sess TO PRINT VALUES TO SCREEN
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)    
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    initial_step = 0
 
    # actual training loop
    training_steps = 1000
    for step in range(initial_step, training_steps):
        sess.run([train_op])
        if step % 100 == 0: # for debug purpose, see how the loss gets decremented thru training steps
            print("loss: ", sess.run([total_loss]))
    evaluate(sess, X, Y)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [0.8389585]
loss:  [0.66806692]
loss:  [0.63865888]
loss:  [0.60634279]
loss:  [0.61393356]
loss:  [0.65270913]
loss:  [0.62827408]
loss:  [0.60302347]
loss:  [0.63809496]
loss:  [0.5576216]
0.71
